# House Prices: Advanced Regression Techniques

In [1]:
# Import modules

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Read data

df_train = pd.read_csv('./Data/train.csv')
df_test = pd.read_csv('./Data/test.csv')

In [3]:
print('Train shape : ',df_train.shape)
print('Test shape  : ',df_test.shape)

Train shape :  (1460, 81)
Test shape  :  (1459, 80)


In [4]:
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## Data Preprocessing

### 1. Handling Missing Values

In [5]:
# Creating a Series of columns and their respective missing value count

missing_values_per_col = df_train.isnull().sum() 
missing_values_per_col

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

## ------ ?????
## What if test set has missing values in columns not captured by the above??
## ------ ?????

In [6]:
# Check for missing values

missing_values_per_col[missing_values_per_col > 0].sort_values(ascending = False)

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
FireplaceQu      690
LotFrontage      259
GarageYrBlt       81
GarageType        81
GarageFinish      81
GarageQual        81
GarageCond        81
BsmtFinType2      38
BsmtExposure      38
BsmtFinType1      37
BsmtCond          37
BsmtQual          37
MasVnrArea         8
MasVnrType         8
Electrical         1
dtype: int64

**1.1 - PoolQC**

In [7]:
# Understanding unique values in training set

df_train['PoolQC'].value_counts()

Gd    3
Ex    2
Fa    2
Name: PoolQC, dtype: int64

In [8]:
# Since there are only a handful, checking which indexes has the above not null values

df_train['PoolQC'][df_train['PoolQC'].notnull()]

197     Ex
810     Fa
1170    Gd
1182    Ex
1298    Gd
1386    Fa
1423    Gd
Name: PoolQC, dtype: object

In [9]:
# Seeing the PoolArea and SalePrice for the not null values of PoolQC

df_train[['PoolQC','PoolArea','SalePrice']][df_train['PoolQC'].notnull()]

,PoolQC,PoolArea,SalePrice
197,Ex,512,235000
810,Fa,648,181000
1170,Gd,576,171000
1182,Ex,555,745000
1298,Gd,480,160000
1386,Fa,519,250000
1423,Gd,738,274970


In [10]:
df_train['PoolArea'].value_counts()

0      1453
738       1
648       1
576       1
555       1
519       1
512       1
480       1
Name: PoolArea, dtype: int64

Based on the above and the description illustrating the possible values of PoolQC, it makes sense that the PoolQC be populated with value "NA" => No Pool for records with a corresponding PoolArea = 0

In [11]:
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer

In [12]:
# Imputer object for imputing PoolQC

feature_transformer = make_column_transformer(
    (SimpleImputer(strategy='constant', fill_value='NA'), ['PoolQC'])
    ,remainder='drop')#'passthrough')

In [13]:
# Imputing PoolQC

df_train['PoolQC'] = feature_transformer.fit_transform(df_train)

In [14]:
# Checking population of missing values

df_train['PoolQC'].value_counts()

NA    1453
Gd       3
Fa       2
Ex       2
Name: PoolQC, dtype: int64

---

**1.2 - MiscFeature**

In [15]:
df_train['MiscFeature'].value_counts()

Shed    49
Gar2     2
Othr     2
TenC     1
Name: MiscFeature, dtype: int64

In [10]:
df_train['MiscVal'].value_counts()

0        1408
400        11
500         8
700         5
450         4
2000        4
600         4
1200        2
480         2
1150        1
800         1
15500       1
620         1
3500        1
560         1
2500        1
1300        1
1400        1
350         1
8300        1
54          1
Name: MiscVal, dtype: int64

In [12]:
df_train['MiscFeature'].notnull()

0       False
1       False
2       False
3       False
4       False
        ...  
1455    False
1456    False
1457     True
1458    False
1459    False
Name: MiscFeature, Length: 1460, dtype: bool

In [16]:
# Looking at the unique values of MiscVal for a corresponding null MiscFeature

df_train['MiscVal'][df_train['MiscFeature'].notnull() == False].unique()

array([0])

In [ ]:
# 

In [26]:
df_train[['MiscVal','MiscFeature']][df_train['MiscFeature'] == 'Othr']#.unique()

,MiscVal,MiscFeature
705,3500,Othr
873,0,Othr


In [19]:
# convert yr columns to difference b/w that yr column and the yrsold feature. 

In [ ]:
# Dont need the "ID" column